<a href="https://colab.research.google.com/github/Edu963/ichimoku/blob/main/Ichimoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
import yfinance as yf

In [60]:
# Download historical stock price data
df = yf.download("COIN", start="2024-03-11", end="2024-12-31", interval='15m')

[*********************100%%**********************]  1 of 1 completed


In [64]:
def ichimoku_cloud(df, n_delay=26, k_delay=9, lag_span=52):
    tenkan_sen_high = df['High'].rolling(window=n_delay).max()
    tenkan_sen_low = df['Low'].rolling(window=n_delay).min()
    tenkan_sen = (tenkan_sen_high + tenkan_sen_low) / 2
    kijun_sen_high = df['High'].rolling(window=k_delay).max()
    kijun_sen_low = df['Low'].rolling(window=k_delay).min()
    kijun_sen = (kijun_sen_high + kijun_sen_low) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(k_delay)
    lagging_span_high = df['High'].rolling(window=lag_span).max()
    lagging_span_low = df['Low'].rolling(window=lag_span).min()
    senkou_span_b = ((lagging_span_high + lagging_span_low) / 2).shift(lag_span)
    chikou_span = df['Close'].shift(-k_delay)
    return pd.DataFrame({
        'Tenkan-Sen': tenkan_sen,
        'Kijun-Sen': kijun_sen,
        'Senkou Span A': senkou_span_a,
        'Senkou Span B': senkou_span_b,
        'Chikou Span': chikou_span,
        'Close': df['Close'],
        'Open': df['Open'],
        'High': df['High'],
        'Low': df['Low']
    }, index=df.index)
ichimoku = ichimoku_cloud(df)

print("Tenkan-Sen:\n", ichimoku['Tenkan-Sen'].head(10))
print("\nKijun-Sen:\n", ichimoku['Kijun-Sen'].head(10))
print("\nSenkou Span A:\n", ichimoku['Senkou Span A'].head(10))
print("\nSenkou Span B:\n", ichimoku['Senkou Span B'].head(10))
print("\nChikou Span:\n", ichimoku['Chikou Span'].head(10))

    #return ichimoku


Tenkan-Sen:
 Datetime
2024-03-11 09:30:00   NaN
2024-03-11 09:45:00   NaN
2024-03-11 10:00:00   NaN
2024-03-11 10:15:00   NaN
2024-03-11 10:30:00   NaN
2024-03-11 10:45:00   NaN
2024-03-11 11:00:00   NaN
2024-03-11 11:15:00   NaN
2024-03-11 11:30:00   NaN
2024-03-11 11:45:00   NaN
Name: Tenkan-Sen, dtype: float64

Kijun-Sen:
 Datetime
2024-03-11 09:30:00           NaN
2024-03-11 09:45:00           NaN
2024-03-11 10:00:00           NaN
2024-03-11 10:15:00           NaN
2024-03-11 10:30:00           NaN
2024-03-11 10:45:00           NaN
2024-03-11 11:00:00           NaN
2024-03-11 11:15:00           NaN
2024-03-11 11:30:00    263.574997
2024-03-11 11:45:00    263.574997
Name: Kijun-Sen, dtype: float64

Senkou Span A:
 Datetime
2024-03-11 09:30:00   NaN
2024-03-11 09:45:00   NaN
2024-03-11 10:00:00   NaN
2024-03-11 10:15:00   NaN
2024-03-11 10:30:00   NaN
2024-03-11 10:45:00   NaN
2024-03-11 11:00:00   NaN
2024-03-11 11:15:00   NaN
2024-03-11 11:30:00   NaN
2024-03-11 11:45:00   NaN
Name:

In [65]:
#ichimoku = ichimoku_cloud(df, displacement=26)

TypeError: ichimoku_cloud() got an unexpected keyword argument 'displacement'

In [66]:
def ichimoku_trading_signals(df, ichimoku, displacement=26):
    buy_signal = (
        (df['Close'].shift(-displacement) > ichimoku['Senkou Span B']) &
        (np.minimum(df['Close'], df['Open']) > ichimoku['Kijun-Sen']) &
        (df['Close'].shift(-displacement) > ichimoku['Senkou Span B'].shift(-displacement))
    )
    sell_signal = (
        (df['Close'].shift(-displacement) < ichimoku['Senkou Span B']) &
        (np.minimum(df['Close'], df['Open']) < ichimoku['Kijun-Sen']) &
        (df['Close'].shift(-displacement) < ichimoku['Senkou Span B'].shift(-displacement))
    )
    signals = pd.DataFrame({
        'Buy Signal': buy_signal,
        'Sell Signal': sell_signal
    }, index=df.index)
    return signals





In [67]:
# Calculate Ichimoku Cloud
ichimoku = ichimoku_cloud(df)

# Generate trading signals
signals = ichimoku_trading_signals(df, ichimoku, displacement=26)

In [68]:
print("Buy signals:", signals['Buy Signal'].sum())
print("Sell signals:", signals['Sell Signal'].sum())

Buy signals: 145
Sell signals: 281


In [69]:
def backtest(df, signals):
    df['Position'] = np.where(signals['Buy Signal'], 1, 0)
    df['Position'] = np.where(signals['Sell Signal'], -1, df['Position'])
    df['Position'] = df['Position'].ffill().fillna(0)
    df['PnL'] = df['Position'].shift(1) * df['Close'].pct_change()
    df['Cumulative PnL'] = df['PnL'].cumsum()
    return df['Cumulative PnL']

cumulative_pnl = backtest(df, signals)
print(cumulative_pnl.tail())

Datetime
2024-05-01 14:45:00    0.304949
2024-05-01 15:00:00    0.304949
2024-05-01 15:15:00    0.304949
2024-05-01 15:30:00    0.304949
2024-05-01 15:45:00    0.304949
Name: Cumulative PnL, dtype: float64


In [72]:
def evaluate_performance(df, signals):
    # Calculate strategy returns
    df['Position'] = np.where(signals['Buy Signal'], 1, np.where(signals['Sell Signal'], -1, 0))
    df['Position'] = df['Position'].ffill().fillna(0)
    df['Returns'] = df['Close'].pct_change()
    df['Strategy Returns'] = df['Position'].shift(1) * df['Returns']

    # Calculate cumulative returns for the strategy
    df['Cumulative Strategy Returns'] = (1 + df['Strategy Returns']).cumprod() - 1

    # Calculate performance metrics
    total_return = df['Cumulative Strategy Returns'].iloc[-1]
    max_drawdown = (df['Cumulative Strategy Returns'].cummax() - df['Cumulative Strategy Returns']).max()
    annualized_sharpe_ratio = (df['Strategy Returns'].mean() / df['Strategy Returns'].std()) * np.sqrt(252)

    trades = signals['Buy Signal'].sum() + signals['Sell Signal'].sum()
    average_profit_loss_per_trade = df['Strategy Returns'].sum() / trades if trades != 0 else 0

    performance_metrics = {
        'Total Return': f'{total_return * 100:.2f}%',
        'Max Drawdown': f'{max_drawdown * 100:.2f}%',
        'Annualized Sharpe Ratio': f'{annualized_sharpe_ratio:.2f}',
        'Number of Trades': trades,
        'Average Profit/Loss per Trade': f'{average_profit_loss_per_trade * 100:.2f}%'
    }

    return performance_metrics



In [73]:
performance_metrics = evaluate_performance(df, signals)
print(performance_metrics)

{'Total Return': '33.34%', 'Max Drawdown': '20.60%', 'Annualized Sharpe Ratio': '0.84', 'Number of Trades': 426, 'Average Profit/Loss per Trade': '0.07%'}


In [57]:
def analyze_performance(df, signals):
    df['Position'] = np.where(signals['Buy Signal'], 1, 0)
    df['Position'] = np.where(signals['Sell Signal'], -1, df['Position'])
    df['Position'] = df['Position'].diff().fillna(0)

    df['Returns'] = df['Close'].pct_change()
    df['Strategy Returns'] = df['Position'].shift(1) * df['Returns']

    total_return = (df['Close'][-1] / df['Close'][0]) - 1
    max_drawdown = (df['Close'] / df['Close'].cummax() - 1).min()
    annualized_sharpe_ratio = np.sqrt(252) * df['Strategy Returns'].mean() / df['Strategy Returns'].std()
    annualized_return = ((1 + total_return) ** (252 / len(df))) - 1
    number_of_trades = signals[signals != 0].sum()
    average_profit_loss_per_trade = df['Strategy Returns'].sum() / number_of_trades

    performance_metrics = {
        'Total Return': f'{total_return * 100:.2f}%',
        'Max Drawdown': f'{max_drawdown * 100:.2f}%',
        'Annualized Sharpe Ratio': f'{annualized_sharpe_ratio:.2f}',
        'Annualized Return': f'{annualized_return * 100:.2f}%',
        'Number of Trades': number_of_trades,
        'Average Profit/Loss per Trade': f'{average_profit_loss_per_trade.values.item() * 100:.2f}%'
    }

    return performance_metrics




In [58]:
performance_metrics = analyze_performance(df, signals)
print(performance_metrics)

ValueError: can only convert an array of size 1 to a Python scalar